# DSE 3260
## Week - 3
### Reg. No - 200968216
#### Pratinav Seth 

<b>The Game:</b><br>
According to the “Six Degrees of Kevin Bacon” game, anyone in the Hollywood film industry can be connected to Kevin Bacon within six steps, where each step consists of finding a film that two actors both starred in. To solve the problem, find the shortest path between any two actors by choosing a sequence of movies that connects them. For example, the shortest path between Jennifer Lawrence and Tom Hanks is 2: Jennifer Lawrence is connected to Kevin Bacon by both starring in “X-Men:  First Class,” and Kevin Bacon is connected to Tom Hanks by both starring in “Apollo 13.”<br><br>
<b>Problem Solving Agent:</b><br>
Given two actors nodes in the graph we need to find the distance (shortest path) between the nodes. Write a python program to determine how many “degrees of separation” apart two actors are. Find the distance or the degree of separation., using

1.   Breadth first search
2.   Depth first search



In [1]:
!gdown https://cdn.cs50.net/ai/2020/x/projects/0/degrees.zip
!unzip /content/degrees.zip
!rm /content/degrees.zip

Downloading...
From: https://cdn.cs50.net/ai/2020/x/projects/0/degrees.zip
To: /content/degrees.zip
100% 22.7M/22.7M [00:00<00:00, 81.9MB/s]
Archive:  /content/degrees.zip
   creating: degrees/
   creating: degrees/large/
  inflating: degrees/large/stars.csv  
  inflating: degrees/large/people.csv  
  inflating: degrees/large/movies.csv  
  inflating: degrees/util.py         
  inflating: degrees/degrees.py      
   creating: degrees/small/
  inflating: degrees/small/stars.csv  
  inflating: degrees/small/people.csv  
  inflating: degrees/small/movies.csv  


In [2]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

We copy util.py for ease of usage. 

In [3]:
class StackFrontier():
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [4]:
class QueueFrontier(StackFrontier):

    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [5]:
import csv
import sys

names = {}
people = {}
movies = {}

We copy degree.py for ease of usage. 

In [6]:
def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass

We make changes to distribution code inorder to incorpate multiple search algorithms.

In [7]:
def main():
    directory = '/content/degrees/small'
    print("Loading data........................")
    load_data(directory)
    print("Data loaded.......................")
    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")
    print("\n********************\n")
    print("Implementing bfs for finding degrees of separation and the path:")
    print("------")
    path = bfs_path(source, target)
    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")
    
    print("\n************\n")
    print("Implementing dfs for finding degrees of separation and the path:")
    print("***********")
    path = dfs_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")

In [8]:
def bfs_path(source, target):
    if source == None:
      return None

    if source == target:
      return []

    start = Node(source, None, None)
    frontier  = QueueFrontier()
    frontier.add(start)
    exploredset = set()

    while True:
      if frontier.empty():
        print('Frontier is Empty - No Connection/Path Between Actors was found!')
        print(len(exploredset), 'actors were explored with no solution/path found between the source and target!')
        return None
        
      frontnode = frontier.remove()
      exploredset.add(frontnode.state)
      neighbors = neighbors_for_person(frontnode.state)

      for action, state in neighbors:

        if frontier.contains_state(state) or state in exploredset:
          continue
        
        if state == target:
          path = []
          path.append((action, state))

          while frontnode.parent != None:
            path.append((frontnode.action, frontnode.state))
            frontnode = frontnode.parent

          path.reverse()

          return path

        newnode = Node(state, frontnode, action)
        frontier.add(newnode)

In [9]:
def dfs_path(source, target):
    if source == None:
      return None

    if source == target:
      return []

    start = Node(source, None, None)
    frontier  = StackFrontier()
    frontier.add(start)
    exploredset = set()

    while True:

      if frontier.empty():
        print('Frontier is Empty - No Connection/Path Between Actors was found!')
        print(len(exploredset), 'actors were explored with no solution/path found between the source and target!')
        return None
            
      frontnode = frontier.remove()
      exploredset.add(frontnode.state)
      neighbors = neighbors_for_person(frontnode.state)

      for action, state in neighbors:
        
        if frontier.contains_state(state) or state in exploredset:
          continue
            
        if not frontier.contains_state(state) and state not in exploredset:
          child = Node(state, frontnode, action)

          if child.state == target:
            path = []
            while child.parent is not None:
              path.append((child.action, child.state))
              child = child.parent
            path.reverse()
            return path
          else:
            frontier.add(child)

In [10]:
def person_id_for_name(name):
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]

In [11]:
def neighbors_for_person(person_id):
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
      for person_id in movies[movie_id]["stars"]:
        neighbors.add((movie_id, person_id))
    return neighbors

In [15]:
main()

Loading data........................
Data loaded.......................
Name: Kevin Bacon
Name: Tom Cruise

********************

Implementing bfs for finding degrees of separation and the path:
------
1 degrees of separation.
1: Kevin Bacon and Tom Cruise starred in A Few Good Men

************

Implementing dfs for finding degrees of separation and the path:
***********
1 degrees of separation.
1: Kevin Bacon and Tom Cruise starred in A Few Good Men


We observe that Breadth first search found the path between the source and target as 2, whereas Depth first search kept exploring the same search tree and found the target at a far greater depth (1026). This clearly shows that Breadth First Search gives the far more optimal path/ solution for a state space search and than Depth First search. This is becuase Breadth first search searches for the states at the same level while Depth first search continues to explore the same search tree until it either find the solution or reaches a node whih has not successors which the search has not previously seen